#MERGE

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split

DATA_FOLDER = "../../DATA/"

merging the shodan and the train set consumes all the ram.

In [7]:
df = pd.read_parquet(DATA_FOLDER + '/PROCESSING/NEW_TRAIN_FEATURES.parq')


In [8]:
df, SAVE = train_test_split(df, test_size=0.5, random_state=0)
SAVE.to_parquet(DATA_FOLDER + 'DATA_JUGGLING/SAVE_MERGE_AUX.parq', index=True)
SAVE
del SAVE

In [9]:
shodan = pd.read_parquet(DATA_FOLDER + "/PROCESSING/NEW_SHODAN_FEATURES.parq")
shodan

,attacker_ip_enum,port_count,HAS_TCP?,HAS_UDP?,risky_ports
0,5915,0,False,False,False
1,3325,1,False,False,True
2,8416,0,False,False,False
3,1213,0,False,False,False
4,9185,0,False,False,False
...,...,...,...,...,...
197674,196812,2,False,False,False
197675,191141,0,False,False,False
197676,195077,3,False,False,True
197677,198002,0,False,False,False


In [10]:
df = pd.merge(df, shodan, left_on='attacker_ip_enum', right_on='attacker_ip_enum', how='left')
df.to_parquet(DATA_FOLDER + 'DATA_JUGGLING/SAVE_MERGE_AUX2.parq', index=True)
del df


In [11]:
SAVE = pd.read_parquet(DATA_FOLDER + "DATA_JUGGLING/SAVE_MERGE_AUX.parq")
SAVE = pd.merge(SAVE, shodan, left_on='attacker_ip_enum', right_on='attacker_ip_enum', how='left')

In [12]:
del shodan

In [13]:
df = pd.read_parquet(DATA_FOLDER + "DATA_JUGGLING/SAVE_MERGE_AUX2.parq")
df = pd.concat([df, SAVE], ignore_index=False)
df

,attack_time,watcher_country,watcher_as_num,watcher_as_name,attacker_country,attacker_as_num,attacker_as_name,attack_type,watcher_uuid_enum,attacker_ip_enum,label,protocol,joined_countries,port_count,HAS_TCP?,HAS_UDP?,risky_ports
0,1.762414,US,-0.091011,PEGTECHINC,FR,-0.502362,OVH SAS,bruteforce,-0.958130,95867,0,ssh,FR:US,3,False,False,True
1,1.409984,PL,-0.503736,home.pl S.A.,FR,-0.502362,OVH SAS,spam,-0.101544,115212,0,http,FR:PL,4,False,False,True
2,0.235214,US,-0.090606,IMH-IAD,IL,-0.596048,Bezeq- THE ISRAEL TELECOMMUNICATION CORP. LTD.,exploit,0.969242,106785,0,http,IL:US,0,False,False,False
3,-1.291987,US,-0.311877,LIQUIDWEB,US,-0.217067,"Alibaba US Technology Co., Ltd.",scan,-0.791761,6738,0,http,US:US,0,False,False,False
4,0.587645,DE,-0.153113,IP-Projects GmbH & Co. KG,FR,-0.450669,Host Europe GmbH,exploit,-0.913167,117158,0,http,FR:DE,6,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20007007,-0.704602,DE,-0.286743,Neue Medien Muennich GmbH,ID,-0.587111,PT Telekomunikasi Indonesia,exploit,-0.735038,173996,0,http,ID:DE,0,False,False,False
20007008,-1.644418,DK,-0.121954,One.com A/S,US,-0.583528,MICROSOFT-CORP-MSN-AS-BLOCK,spam,0.228971,35213,0,http,US:DK,17,False,False,True
20007009,-0.352171,NL,-0.238176,CJ2 Hosting B.V.,SG,0.741896,Contabo Asia Private Limited,scan,-0.569887,87392,0,http,SG:NL,1,False,False,True
20007010,-1.761895,FR,1.379804,GANDI SAS,SG,-0.524284,DIGITALOCEAN-ASN,scan,-0.904705,22588,0,http,SG:FR,0,False,False,False


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 40014024 entries, 0 to 20007011
Data columns (total 17 columns):
 #   Column             Dtype   
---  ------             -----   
 0   attack_time        float64 
 1   watcher_country    category
 2   watcher_as_num     float32 
 3   watcher_as_name    category
 4   attacker_country   category
 5   attacker_as_num    float32 
 6   attacker_as_name   category
 7   attack_type        category
 8   watcher_uuid_enum  float64 
 9   attacker_ip_enum   int64   
 10  label              int8    
 11  protocol           category
 12  joined_countries   category
 13  port_count         int64   
 14  HAS_TCP?           bool    
 15  HAS_UDP?           bool    
 16  risky_ports        bool    
dtypes: bool(3), category(7), float32(2), float64(2), int64(2), int8(1)
memory usage: 2.4 GB


#SAVING

In [15]:
def fill_nan_with_mean(df):
    for column in df.columns:
        if pd.api.types.is_numeric_dtype(df[column]):
            mean_value = df[column].mean()
            df[column].fillna(mean_value, inplace=True)
        elif pd.api.types.is_categorical_dtype(df[column]):
            mode_value = df[column].mode().iloc[0]  # Mode for categorical columns
            df[column].fillna(mode_value, inplace=True)

fill_nan_with_mean(df)

C:\Users\Francisco\AppData\Local\Temp\ipykernel_1240\2981802581.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna(mean_value, inplace=True)
C:\Users\Francisco\AppData\Local\Temp\ipykernel_1240\2981802581.py:6: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  elif pd.api.types.is_categorical_dtype(df[column]):
C:\Users\Francisco\AppData\Local\Temp\ipykernel_1240\2981802581.py:8: FutureWarning: A value is tryin

In [16]:
df.to_parquet(DATA_FOLDER + '/PROCESSING/train_models_merged_features.parq', index=True)